# Homework Assignment 5: Model Evaluation
As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will utilize a copy of the extracted feature dataset we have been working with. The dataset has been processed by performing outlier clipping, z-score and range scaling, and forward feature selection to select 20 features. We are now going to utilize more than one partition worth of data, so for the z-score and range scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 

---

## Step 1: Downloading the Data

This assignment will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) and will add the use of [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---

For this assignment, cleaning, transforming, and normalization of the data has been completed using both partitions to find the various minimum, maximum, standard deviation, and mean values needed to perform these operations. Recall from lecture that we should not perform these operations on each partition individually, but as a whole as there may(will) be different values for these in different partitions. 

For example, if we perform simple range scaling on each partition individually and we see a range of 0 to 100 in one partition and 0 to 10 in another. After individual scaling the values with 100 in the first would be mapped to 1 just like the values that had 10 in the second. This can cause serious performance problems in your model, so I have made sure that the normalization was treated properly for you. 

Below you will find the full partitions and `toy` sampled data from each partition, where only 20 samples from each of our 5 classes have been included in the data.  

#### Full
- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Full Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2ExtractedFeatures.csv)

#### Toy
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition2ExtractedFeatures.csv)

Now that you have the two files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

For each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

In [16]:
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
def calc_tss(y_true=None, y_predict=None):
    """
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    """
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

As in the previous assignment, we will be utilizing a binary classification of our 5 class dataset. So, below is the helper function to change our class labels from the 5 class target feature to the binary target feature. The function is implemented to take a dataframe (e.g. our `abt`) and prepares it for a binary classification by merging the `X`- and `M`-class samples into one group, and the rest (`NF`, `B`, and `C`) into another group, labeled with `1`s and `0`s, respectively.

In [3]:
def dichotomize_X_y(data: pd.DataFrame):
    """
    dichotomizes the dataset and split it into the features (X) and the labels (y).
    
    :return: two np.ndarray objects X and y.
    """
    data_dich = data.copy()
    data_dich['lab'] = data_dich['lab'].map({'NF': 0, 'B': 0, 'C': 0, 'M': 1, 'X': 1})
    y = data_dich['lab']
    X = data_dich.drop(['lab'], axis=1)
    return X.values, y.values

In [4]:
data_dir = '/Users/npunj/Documents/Fund_Data_Science/Hw5'
data_file = "normalized_partition1ExtractedFeatures.csv"
data_file2 = "normalized_partition2ExtractedFeatures.csv"

In [5]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2))

---
### Q1 (10 points)

Just like you did with the previous assignment, you will be utilizing a few different types of feature selection to find subsets of descriptive features to use in the models we will be evaluating.  For this question you will again be utilizing the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). You will then be using 3 diferent feature evaluation functions.

-  [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif)

- [scikit-learn mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

- [chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2)

For each of these combinations of evaluation functions, you need to construct a 20 feature training and testing dataset. This will be done by:
<ol>
    <li>Use the `SelectKBest` class with each of the evaluation functions to perform feature selection using Partition 1 as your input data</li>
    <li>Construct a new train `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 1</li>
    <li>Construct a new test `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 2</li>
</ol>

After this question, you should have a total of 6 `DataFrame`s to use in later questions, a train and test pair for each feature selection method.

---

In [7]:
numFeat = 20
abt_cpy = abt.copy()
abt2_cpy = abt2.copy()

In [11]:
# split train and test data from partition1, training set, and partition2, testing set
X_train = abt_cpy.drop(['lab'], axis=1)
y_train = abt_cpy['lab']
X_test = abt2_cpy.drop(['lab'], axis=1)
y_test = abt2_cpy['lab']

#Use SelectKBest to create Dataframes for train and test models
selector = SelectKBest(f_classif, k=numFeat)
selector2 = SelectKBest(f_classif, k=numFeat)
new_select = selector.fit_transform(X_train,y_train)
new_select2 = selector2.fit_transform(X_test,y_test)
select_feats = list(np.take((X_train.columns), np.where(selector.get_support()==True)[0]))
select_feats2 = list(np.take((X_test.columns), np.where(selector.get_support()==True)[0]))
temp_train_df = pd.DataFrame(new_select, columns = select_feats)
temp_test_df = pd.DataFrame(new_select2, columns = select_feats2)
y_db = pd.DataFrame(y_train)
y2_db = pd.DataFrame(y_test)

fclass_train1 = y_db.join(temp_train_df)
fclass_test = y2_db.join(temp_test_df)

selector = SelectKBest(mutual_info_classif, k=numFeat)
selector2 = SelectKBest(mutual_info_classif, k=numFeat)
new_select = selector.fit_transform(X_train,y_train)
new_select2 = selector2.fit_transform(X_test,y_test)
select_feats = list(np.take((X_train.columns), np.where(selector.get_support()==True)[0]))
select_feats2 = list(np.take((X_test.columns), np.where(selector.get_support()==True)[0]))
temp_train_df = pd.DataFrame(new_select, columns = select_feats)
temp_test_df = pd.DataFrame(new_select2, columns = select_feats2)
y_db = pd.DataFrame(y_train)
y2_db = pd.DataFrame(y_test)

mutaul_info_train = y_db.join(temp_train_df)
mutaul_info_test = y2_db.join(temp_test_df)

selector = SelectKBest(chi2, k=numFeat)
selector2 = SelectKBest(chi2, k=numFeat)
new_select = selector.fit_transform(X_train,y_train)
new_select2 = selector2.fit_transform(X_test,y_test)
select_feats = list(np.take((X_train.columns), np.where(selector.get_support()==True)[0]))
select_feats2 = list(np.take((X_test.columns), np.where(selector.get_support()==True)[0]))
temp_train_df = pd.DataFrame(new_select, columns = select_feats)
temp_test_df = pd.DataFrame(new_select2, columns = select_feats2)
y_db = pd.DataFrame(y_train)
y2_db = pd.DataFrame(y_test)

chi_train = y_db.join(temp_train_df)
chi_test = y2_db.join(temp_test_df)

    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

In [12]:
display(fclass_train1)
display(fclass_test)

,lab,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTUSJH_mean,TOTUSJH_linear_weighted_average,TOTUSJH_quadratic_weighted_average,TOTUSJH_last_value,ABSNJZH_max,ABSNJZH_stddev,...,ABSNJZH_gderivative_stddev,ABSNJZH_average_absolute_change,ABSNJZH_average_absolute_derivative_change,ABSNJZH_avg_mono_increase_slope,SAVNCPP_avg_mono_decrease_slope,SAVNCPP_avg_mono_increase_slope,SAVNCPP_dderivative_stddev,SAVNCPP_average_absolute_change,SAVNCPP_gderivative_stddev,SAVNCPP_average_absolute_derivative_change
0,NF,0.238758,0.275990,0.244674,0.249912,0.249364,0.250146,0.271143,0.129462,0.166475,...,0.166540,0.146038,0.138859,0.147190,0.917760,0.071006,0.080273,0.076320,0.111590,0.065199
1,NF,0.106759,0.123894,0.114227,0.114189,0.111154,0.109375,0.108247,0.081670,0.122735,...,0.108461,0.096518,0.091213,0.101681,0.973846,0.023172,0.025106,0.025291,0.031024,0.022525
2,NF,0.116361,0.141522,0.126501,0.128604,0.125150,0.123883,0.120471,0.092963,0.105429,...,0.118634,0.111193,0.106288,0.108718,0.966885,0.029407,0.029272,0.028676,0.035628,0.023937
3,NF,0.315587,0.328616,0.322563,0.321839,0.322904,0.322843,0.320618,0.170967,0.183616,...,0.167205,0.146319,0.132143,0.147689,0.940147,0.056448,0.060894,0.061220,0.073133,0.054571
4,NF,0.125745,0.140699,0.134145,0.134307,0.133530,0.132812,0.124584,0.117000,0.103106,...,0.115037,0.109309,0.104402,0.104563,0.966184,0.029922,0.032664,0.034008,0.034723,0.031312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73487,NF,0.038092,0.066916,0.057023,0.056257,0.053632,0.052391,0.047050,0.041284,0.057351,...,0.070496,0.065722,0.063366,0.068696,0.985687,0.014996,0.013708,0.012735,0.013920,0.011287
73488,NF,0.086322,0.134423,0.123106,0.120311,0.122114,0.121065,0.113201,0.105800,0.136841,...,0.133634,0.124009,0.118593,0.122110,0.962417,0.031565,0.032351,0.031525,0.036066,0.028234
73489,C,0.409456,0.429922,0.416225,0.417221,0.415499,0.414165,0.416328,0.165755,0.214363,...,0.219139,0.197064,0.182976,0.195310,0.909242,0.103355,0.111763,0.111319,0.117084,0.109788
73490,B,0.370843,0.395401,0.379373,0.382027,0.377390,0.374104,0.366209,0.142870,0.184096,...,0.205729,0.192532,0.185899,0.204119,0.881328,0.107276,0.104509,0.108241,0.111436,0.095988


,lab,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTUSJH_mean,TOTUSJH_linear_weighted_average,TOTUSJH_quadratic_weighted_average,TOTUSJH_last_value,ABSNJZH_max,ABSNJZH_stddev,...,ABSNJZH_gderivative_stddev,ABSNJZH_average_absolute_change,ABSNJZH_average_absolute_derivative_change,ABSNJZH_avg_mono_increase_slope,SAVNCPP_avg_mono_decrease_slope,SAVNCPP_avg_mono_increase_slope,SAVNCPP_dderivative_stddev,SAVNCPP_average_absolute_change,SAVNCPP_gderivative_stddev,SAVNCPP_average_absolute_derivative_change
0,NF,0.248654,0.264179,0.257628,0.257166,0.257336,0.255972,0.253303,0.246850,0.221667,...,0.219492,0.157880,0.175954,0.907985,0.099690,0.100066,0.102358,0.085923,0.087059,0.098160
1,NF,0.148949,0.161652,0.154316,0.154432,0.154392,0.154432,0.152454,0.080530,0.113363,...,0.051378,0.117204,0.121236,0.966180,0.031892,0.034760,0.034980,0.036871,0.026473,0.032019
2,NF,0.161101,0.174518,0.164688,0.165918,0.166817,0.167077,0.163801,0.134300,0.158542,...,0.104512,0.159358,0.163217,0.948398,0.049728,0.051331,0.052093,0.062940,0.057651,0.045289
3,NF,0.263200,0.292981,0.279195,0.278289,0.273310,0.270127,0.260010,0.141124,0.188205,...,0.123901,0.139561,0.138083,0.943507,0.059199,0.058710,0.059671,0.075896,0.070554,0.051173
4,NF,0.091531,0.104345,0.097824,0.098008,0.098146,0.097687,0.094275,0.063610,0.086037,...,0.035699,0.090478,0.089760,0.975876,0.022294,0.025694,0.025180,0.028386,0.017667,0.023367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88552,NF,0.181140,0.205758,0.190417,0.192371,0.188244,0.185497,0.180158,0.108809,0.137283,...,0.062360,0.140601,0.138010,0.939266,0.057497,0.065002,0.062841,0.056108,0.080029,0.061766
88553,NF,0.049787,0.058479,0.054282,0.054346,0.054770,0.055035,0.057382,0.048622,0.058834,...,0.024737,0.071568,0.073404,0.986529,0.014260,0.014678,0.013554,0.018163,0.009257,0.012104
88554,NF,0.047382,0.062495,0.054054,0.054722,0.054027,0.053349,0.053345,0.045427,0.061008,...,0.025985,0.078785,0.079521,0.987869,0.012566,0.012885,0.013014,0.015375,0.013978,0.011461
88555,NF,0.094972,0.115870,0.106425,0.106316,0.108176,0.107972,0.105768,0.101883,0.121610,...,0.072735,0.094667,0.097608,0.973282,0.022278,0.027569,0.024832,0.028041,0.030783,0.023464


---
### Q2 (10 points)

Now that we have our training and testing datasets for each of our feature subsets, we need to attempt to perform hyperparameter tuning on our model for each of the datasets. We want to see which combination of dataset and parameter settings seem to provide the best results. 

In order to do this, we must first dichotomize the training and testing data. Lucky for you, a method has already been provided to do this. All you need to do is apply it to teach of the `DataFrame`s you constructed in Q1.  

With your binary classification dataset constructed, now it's time to start training and testing some models. We will start with the simple [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, for each of your three copies of the Partition 1 training datasets that have had their `lab` columns converted to a binary label, train 4 different instances with the following settings. **(see documentation to know what these are)** In total you will train and evaluate 12 model setting and feature selected data pairings. 

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result. **NOTE: The model does take a little while to evaluate.**

---

In [13]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [67]:
#fclass_train1, fclass_test, mutual_info_train, mutual_info_test, chi_train, chi_test
fclass_train_X, fclass_train_y = dichotomize_X_y(fclass_train1)
fclass_test_X, fclass_test_y = dichotomize_X_y(fclass_test)

mutual_info_train_X, mutual_info_train_y = dichotomize_X_y(mutaul_info_train)
mutual_info_test_X, mutual_info_test_y = dichotomize_X_y(mutaul_info_test)

chi_train_X, chi_train_y = dichotomize_X_y(chi_train)
chi_test_X, chi_test_y = dichotomize_X_y(chi_train)

def get_tss(X_train, y_train, X_test, y_test, n, p_val):
    model = kNN(n_neighbors = n, p = p_val)
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_tss = calc_tss(y_test, y_pred)
    return model_tss

model1 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3, 1)
model2 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3, 2)
model3 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5, 1)
model4 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5, 2)
model5 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3, 1)
model6 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3, 2)
model7 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5, 1)
model8 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5, 2)
model9 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3, 1)
model10 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3, 2)
model11 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5, 1)
model12 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5, 2)
    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=85645	FP=1511	FN=882	TP=519
TN=85107	FP=2049	FN=907	TP=494
TN=85769	FP=1387	FN=890	TP=511
TN=85361	FP=1795	FN=900	TP=501
TN=86112	FP=1044	FN=1099	TP=302
TN=86382	FP=774	FN=1167	TP=234
TN=86276	FP=880	FN=1126	TP=275
TN=86477	FP=679	FN=1185	TP=216
TN=72234	FP=4	FN=7	TP=1247
TN=72227	FP=11	FN=14	TP=1240
TN=72226	FP=12	FN=27	TP=1227
TN=72218	FP=20	FN=42	TP=1212


In [68]:
display(model1)
display(model2)
display(model3)
display(model4)
display(model5)
display(model6)
display(model7)
display(model8)
display(model9)
display(model10)
display(model11)
display(model12)

0.3531129492584269

0.3290957128928679

0.3488254785064817

0.33700645857588984

0.20358179278918317

...

0.14638496056641354

0.9943624903202964

0.9886834512516287

0.9783027819656741

0.9662303144403979

---
### Q3 (10 points)

After evaluating the various results from Q2, you will notice that the results are not all that great with greater than 1000 false negatives for nearly all of our settings tried. But, what can be done to improve our results? If you read the documentation for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), which you certainly should have, you will see that we were only using the `MinkowskiDistance` metric with different values of `p`. If you look into the [DistanceMetric](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric) documentation for the neighbors classifiers, you will see there are several others available to use.

So, for this question, train and evaluate two more instances of [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) for each of our different feature selection train test datsets, but this time using the `ChebyshevDistance` metric instead of the `MinkowskiDistance` metric.  For these models you will only be changing the number neighbors to 3 and 5, as the values of `p` are not used for the `ChebyshevDistance` metric. 

---

In [28]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [29]:
def get_tss2(X_train, y_train, X_test, y_test, n):
    model = kNN(n_neighbors = n, metric = 'chebyshev')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_tss = calc_tss(y_test, y_pred)
    return model_tss

model13 = get_tss2(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3)
model14 = get_tss2(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5)
model15 = get_tss2(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3)
model16 = get_tss2(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5)
model17 = get_tss2(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3)
model18 = get_tss2(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5)

    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=86995	FP=161	FN=1322	TP=79
TN=86976	FP=180	FN=1310	TP=91
TN=87136	FP=20	FN=1395	TP=6
TN=87135	FP=21	FN=1395	TP=6
TN=72215	FP=23	FN=38	TP=1216
TN=72199	FP=39	FN=92	TP=1162


---
### Q4 (10 points)

After evaluating the results from Q3, you will see that the results are no better than those we found for Q2. This leads to the thought that maybe the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) is just not a good fit for the problem we are applying it to. So, let's move on to another classifier for this problem. 

In this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, continuing to use our training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label, train 8 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [30]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [37]:
def get_tss3(X_train, y_train, X_test, y_test, c_val, d_val, s_val):
    model = DecisionTreeClassifier(criterion = c_val, max_depth = d_val, splitter = s_val)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_tss = calc_tss(y_test, y_pred)
    return model_tss

model19 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', 5, 'best')
model20 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', 5, 'random')
model21 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', None, 'best')
model22 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', None, 'random')
model23 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', 5, 'best')
model24 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', 5, 'random')
model25 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', None, 'best')
model26 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', None, 'random')
model27 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', 5, 'best')
model28 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', 5, 'random')
model29 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', None, 'best')
model30 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', None, 'random')
model31 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', 5, 'best')
model32 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', 5, 'random')
model33 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', None, 'best')
model34 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', None, 'random')
model35 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', 5, 'best')
model36 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', 5, 'random')
model37 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', None, 'best')
model38 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', None, 'random')
model39 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', 5, 'best')
model40 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', 5, 'random')
model41 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', None, 'best')
model42 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', None, 'random')
    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=86478	FP=678	FN=1090	TP=311
TN=86898	FP=258	FN=1200	TP=201
TN=84006	FP=3150	FN=702	TP=699
TN=84430	FP=2726	FN=771	TP=630
TN=86504	FP=652	FN=1094	TP=307
TN=86844	FP=312	FN=1172	TP=229
TN=85355	FP=1801	FN=1020	TP=381
TN=84872	FP=2284	FN=954	TP=447
TN=85113	FP=2043	FN=932	TP=469
TN=87037	FP=119	FN=1368	TP=33
TN=81200	FP=5956	FN=806	TP=595
TN=84624	FP=2532	FN=1023	TP=378
TN=85518	FP=1638	FN=788	TP=613
TN=87151	FP=5	FN=1400	TP=1
TN=85518	FP=1638	FN=917	TP=484
TN=81063	FP=6093	FN=661	TP=740
TN=71994	FP=244	FN=616	TP=638
TN=72041	FP=197	FN=812	TP=442
TN=72238	FP=0	FN=0	TP=1254
TN=72238	FP=0	FN=0	TP=1254
TN=72104	FP=134	FN=862	TP=392
TN=72166	FP=72	FN=1060	TP=194
TN=72238	FP=0	FN=0	TP=1254
TN=72238	FP=0	FN=0	TP=1254


In [38]:
display(model19)
display(model20)
display(model21)
display(model22)
display(model23)
display(model24)
display(model25)
display(model26)
display(model27)
display(model28)
display(model29)
display(model30)
display(model31)
display(model32)
display(model33)
display(model34)
display(model35)
display(model36)
display(model37)
display(model38)
display(model39)
display(model40)
display(model41)
display(model42)


0.21420514231146043

0.14050874146955278

0.46278724614300104

0.41840155086800473

0.21164835447782573

...

1.0

0.31074470164699686

0.15370823884348622

1.0

1.0

---
### Q5 (10 points)

After evaluating results from Q4, you will see that the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) was able to accomplish a bit of an improvement over the best resutls we found for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).  This is indeed great, but can we do better than this if we use yet another classifier? Let's move on to yet another and find out.

For this question you will be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. We won't be changing any of the default settings, just train 1 model for each of our feature selected data subsets. You will again be using your training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label. You will then test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [46]:
def get_tss4(X_train, y_train, X_test, y_test):
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    clf_tss = calc_tss(y_test, y_pred)
    return clf_tss

model43 = get_tss4(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y)
model44 = get_tss4(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y)
model45 = get_tss4(chi_train_X, chi_train_y, chi_test_X, chi_test_y)
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=0	FP=87156	FN=0	TP=1401
TN=87156	FP=0	FN=1401	TP=0
TN=63679	FP=8559	FN=81	TP=1173


In [47]:
display(model43)
display(model44)
display(model45)

0.0

0

0.816923351849568

---
### Q6 (10 points)

If you recall from a lecture some time back, it was shown that another way of improving the results of classification is to perform some form of sampling to balance the number of samples there are for the various classes. The reason why this works for specific classifiers, and methods for doing the sampling, are numerious and we don't have enough time to cover all of them in this course.  However, it is still beneficial to know this works and that it is something that you should be considering when you are training models.  

So, for this question, we will implement a very naive method for sampling so we can use the results for training our models again.  Below you will find a function stub, complete the function and have it return a copy of the input dataframe where each class (except for the smallest one) have been undersampled to match the size of the smallest class in the dataset. In this function you should assume the `lab` column is the class label and not the dicotomized binary classification converted label.

To do this you may want to use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function of the DataFrame to get groups of rows from your DataFrame.  You may also wish to use the [sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) function to select a number of rows from a group. You can also use the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method to process each group from your grouped rows. These are just hints, you can solve the problem how you see fit.

Once this function is complete, apply it to each of your training datasets that have been constructed with different feature selection methods from partition 1 (the ones with all the NF, C, .., X labels). You will not be applying this to your testing sets. After you have your sampled feature selected datasets, you will then apply your function that converts the multi-class problem to a binary problem to each of the resultant selected subsets so we can use these new undersampled data for the next several questions.

---

In [48]:
def perform_under_sample(data:DataFrame)->DataFrame:
    df_copy = data.copy()
    group_df = df_copy.groupby(['lab'])
    group_df = group_df.sample(frac = 0.5)
    return group_df
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

In [52]:
#fclass_train1, fclass_test, mutual_info_train, mutual_info_test, chi_train, chi_test
usample_fclass_train = perform_under_sample(fclass_train1)
usample_mutual_info_train = perform_under_sample(mutaul_info_train)
usample_chi_train = perform_under_sample(chi_train)
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

In [57]:
fclass_train_X, fclass_train_y = dichotomize_X_y(usample_fclass_train)
#fclass_test_X, fclass_test_y = dichotomize_X_y(fclass_test)

mutual_info_train_X, mutual_info_train_y = dichotomize_X_y(usample_mutual_info_train)
#mutual_info_test_X, mutual_info_test_y = dichotomize_X_y(mutaul_info_test)

chi_train_X, chi_train_y = dichotomize_X_y(usample_chi_train)
#chi_test_X, chi_test_y = dichotomize_X_y(chi_train)

---
### Q7

For this question repeat what you did for Q2, but with your balanced binary classification datasets constructed in Q6, uese the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. 

So, train 4 different instances with the following settings for each of your feature selected subsets, for a total of 12 different evaluations. **(see documentation to know what these are)**

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced). You shall then calculate and print the TSS score for each result. **NOTE: The model now takes less time to evaluate!**

---

In [60]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [58]:
model46 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3, 1)
model47 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3, 2)
model48 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5, 1)
model49 = get_tss(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5, 2)
model50 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3, 1)
model51 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3, 2)
model52 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5, 1)
model53 = get_tss(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5, 2)
model54 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3, 1)
model55 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3, 2)
model56 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5, 1)
model57 = get_tss(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5, 2)    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=85722	FP=1434	FN=950	TP=451
TN=85462	FP=1694	FN=931	TP=470
TN=85934	FP=1222	FN=960	TP=441
TN=85418	FP=1738	FN=890	TP=511
TN=86263	FP=893	FN=1131	TP=270
TN=86468	FP=688	FN=1178	TP=223
TN=86496	FP=660	FN=1194	TP=207
TN=86472	FP=684	FN=1211	TP=190
TN=72188	FP=50	FN=74	TP=1180
TN=72177	FP=61	FN=80	TP=1174
TN=72174	FP=64	FN=133	TP=1121
TN=72171	FP=67	FN=147	TP=1107


In [59]:
display(model46)
display(model47)
display(model48)
display(model49)
display(model50)
display(model51)
display(model52)
display(model53)
display(model54)
display(model55)
display(model56)
display(model57)

0.30545966311311834

0.31603824808758085

0.3007543243978186

0.34479821704427605

0.18247349039547392

...

0.12776941943575443

0.9402966792429402

0.9353597158215226

0.8930534336414898

0.8818476299303565

---
### Q8

After evaluating the various results from Q7, you will notice that some of the results are improved over the same experiments we conducted in Q2. Additionally, you should also notice a improvement in the speed at which the results were obtained. The question now is will we continue to see these improvements for all of our experiments? So, let's move on and see.

For this question, you will repeat the experiments from Q3, but using the balanced binary classification datasets constructed in Q6. You will still be using the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) like you did in Q7, but you will again be changing from using the `MinkowskiDistance` metric with different values of `p` to using the `ChebyshevDistance` metric. You will construct two models for each of your feature selected datasets by changing the number neighbors to 3 and 5.

Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced), then calculate and print the TSS score for each result. 

---

In [61]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [62]:
model158 = get_tss2(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 3)
model159 = get_tss2(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 5)
model160 = get_tss2(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 3)
model161 = get_tss2(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 5)
model162 = get_tss2(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 3)
model162 = get_tss2(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 5)    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=87014	FP=142	FN=1318	TP=83
TN=87000	FP=156	FN=1291	TP=110
TN=87136	FP=20	FN=1395	TP=6
TN=87137	FP=19	FN=1395	TP=6
TN=72164	FP=74	FN=140	TP=1114
TN=72135	FP=103	FN=230	TP=1024


---
### Q9

After evaluating the results of Q8 things are looking a little less encouraging, since none of those results look to be better than the results of Q7. However, the results from Q3 weren't really any better than Q2 in the first place, so not all is lost.  Let's continue on and see how things turn out with models like we used in Q4 since those were actaully an improvement over Q2 originally.

So, in this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), like you did in Q4, and try several different settings to see how/if using different settings will improve our score. The difference will again be that you are now using the balanced binary classification datasets constructed in Q6 to train 8 different instances for each of your feature selected datasets using the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score for each result. 

---

In [63]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [64]:
model63 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', 5, 'best')
model64 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', 5, 'random')
model65 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', None, 'best')
model66 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'gini', None, 'random')
model67 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', 5, 'best')
model68 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', 5, 'random')
model69 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', None, 'best')
model70 = get_tss3(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y, 'entropy', None, 'random')
model71 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', 5, 'best')
model72 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', 5, 'random')
model73 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', None, 'best')
model74 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'gini', None, 'random')
model75 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', 5, 'best')
model76 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', 5, 'random')
model77 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', None, 'best')
model78 = get_tss3(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y, 'entropy', None, 'random')
model79 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', 5, 'best')
model80 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', 5, 'random')
model81 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', None, 'best')
model82 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'gini', None, 'random')
model83 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', 5, 'best')
model84 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', 5, 'random')
model85 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', None, 'best')
model86 = get_tss3(chi_train_X, chi_train_y, chi_test_X, chi_test_y, 'entropy', None, 'random')    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=86593	FP=563	FN=1259	TP=142
TN=86994	FP=162	FN=1360	TP=41
TN=83602	FP=3554	FN=685	TP=716
TN=84690	FP=2466	FN=841	TP=560
TN=86286	FP=870	FN=1086	TP=315
TN=86815	FP=341	FN=1196	TP=205
TN=84006	FP=3150	FN=840	TP=561
TN=85267	FP=1889	FN=1009	TP=392
TN=86152	FP=1004	FN=1088	TP=313
TN=86869	FP=287	FN=1290	TP=111
TN=83094	FP=4062	FN=910	TP=491
TN=82283	FP=4873	FN=989	TP=412
TN=86864	FP=292	FN=1289	TP=112
TN=87107	FP=49	FN=1382	TP=19
TN=84967	FP=2189	FN=916	TP=485
TN=84796	FP=2360	FN=1030	TP=371
TN=72054	FP=184	FN=643	TP=611
TN=72077	FP=161	FN=819	TP=435
TN=72145	FP=93	FN=105	TP=1149
TN=72107	FP=131	FN=103	TP=1151
TN=72057	FP=181	FN=810	TP=444
TN=72026	FP=212	FN=860	TP=394
TN=72142	FP=96	FN=103	TP=1151
TN=72120	FP=118	FN=113	TP=1141


In [65]:
display(model63)
display(model64)
display(model65)
display(model66)
display(model67)
display(model68)
display(model69)
display(model70)
display(model71)
display(model72)
display(model73)
display(model74)
display(model75)
display(model76)
display(model77)
display(model78)
display(model79)
display(model80)
display(model81)
display(model82)
display(model83)
display(model84)
display(model85)
display(model86)

0.09489649267065292

0.027406074789913738

0.4702860695380643

0.37142039630039436

0.21485729936809755

...

0.916049388930698

0.35156137917842284

0.3112598338656646

0.9165338984686142

0.9082548679575175

---
### Q10

Unlike with [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), it seems that the sampling didn't really help much for the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).  Where before we saw a 3X improvement with the Decision Tree over the KNN classifier, we now see similar results for both classifiers.  Let's see how switching to the sampled data affectes our best performing classifier when we were using the full dataset.

For this question you will again be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier as you did in Q5 but using your balanced binary classification dataset constructed in Q6 to train just 1 model for each feature selected dataset. Once you have done that, test the model using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score. 

---

In [66]:
model87 = get_tss4(fclass_train_X, fclass_train_y, fclass_test_X, fclass_test_y)
model88 = get_tss4(mutual_info_train_X, mutual_info_train_y, mutual_info_test_X, mutual_info_test_y)
model89 = get_tss4(chi_train_X, chi_train_y, chi_test_X, chi_test_y)    
    #----------------------------------------------
    # TODO: Complete here.
    #----------------------------------------------

TN=0	FP=87156	FN=0	TP=1401
TN=87156	FP=0	FN=1401	TP=0
TN=63387	FP=8851	FN=71	TP=1183


Unfortunately, we don't see much improvement for our [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. 

**Note:The TA would like you to turn in assignments that have been run and have results, so make sure to do a restart and run all from the kernel menu. Then make sure to save before you turn it in. You might find it necessary to use the toy dataset if you have time constraints.**